In [ ]:
pip install -r requirements.txt

In [ ]:
# Run this in your Jupyter notebook cell
#import sys
#import subprocess

# Install in the current Python environment
#subprocess.check_call([sys.executable, "-m", "pip", "install", "sentence-transformers", "torch"])

0

# Cell 1: Import Libraries

In [1]:
import mysql.connector
import pandas as pd
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import os
import time

# Cell 2: Database Connection

In [2]:
# Step 1: Database Connection
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='YOUR SQL PASSWORD',
    database='campervan'
)
print("Database connected successfully!")

Database connected successfully!


# Cell 3: Vector Store Setup Function

Create a folder rag_data/ with:

schema_description.txt

sql_examples.txt 

business_rules.txt

In [3]:
# Step 2: Load and process documents for RAG (with error handling)
def setup_vector_store():
    """Setup vector store with SQL examples and schema info"""
    try:
        # Check if file exists first
        file_path = "rag_data/sql_examples.txt"
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        
        # Load SQL examples and schema/business rules
        print("🔄 Loading documents...")
        loader = TextLoader(file_path, encoding='utf-8')
        docs = loader.load()
        print(f"✅ Loaded {len(docs)} document(s).")
        
        if len(docs) == 0:
            raise ValueError("No documents were loaded!")
        
        # Print first document info
        print(f"📄 Document content length: {len(docs[0].page_content)} characters")
        print("📋 First 200 characters:")
        print(docs[0].page_content[:200])
        print("...")
        
        # Split documents into chunks
        print("🔄 Splitting documents into chunks...")
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=20,
            separators=["\n\n", "\n", ".", " ", ""]
        )
        chunks = splitter.split_documents(docs)
        print(f"✅ Split into {len(chunks)} chunks.")
        
        # Print sample chunk
        if len(chunks) > 0:
            print(f"📋 Sample chunk (first 200 chars):")
            print(chunks[0].page_content[:200])
            print("...")
        
        # Create embeddings
        print("🔄 Creating embeddings...")
        embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        print("✅ Embedding model loaded.")
        
        # Create vector store
        print("🔄 Creating vector store...")
        db = Chroma.from_documents(
            documents=chunks,
            embedding=embedding_model,
            persist_directory="vector_db"
        )
        print("✅ Vector DB created and persisted.")
        
        # Test the vector store
        print("🔄 Testing vector store...")
        test_query = "expenses by department"
        results = db.similarity_search(test_query, k=2)
        print(f"✅ Vector store test successful! Found {len(results)} similar documents.")
        print(f"📋 Sample result: {results[0].page_content[:100]}...")
        
        return db
        
    except Exception as e:
        print(f"❌ Error in setup_vector_store: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Test the vector store setup
print("Starting vector store setup...")
print("=" * 50)

db = setup_vector_store()

if db:
    print("=" * 50)
    print("🎉 SUCCESS! Vector store created successfully!")
    print("✅ You can proceed to the next cells.")
else:
    print("=" * 50)
    print("❌ FAILED! Please check the errors above and fix them.")

Starting vector store setup...
🔄 Loading documents...
✅ Loaded 1 document(s).
📄 Document content length: 45733 characters
📋 First 200 characters:


-- 1. Q: Show all employees in the IT & System department
SELECT employee_id, employee_name, role, hire_date
FROM dim_employee
WHERE department = 'IT & System';

-- 2. Q: Find all expense categories
...
🔄 Splitting documents into chunks...
✅ Split into 115 chunks.
📋 Sample chunk (first 200 chars):
-- 1. Q: Show all employees in the IT & System department
SELECT employee_id, employee_name, role, hire_date
FROM dim_employee
WHERE department = 'IT & System';

-- 2. Q: Find all expense categories t
...
🔄 Creating embeddings...


c:\Users\ABHINAVVIKRAM\anaconda3\envs\PROJECT_ENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Embedding model loaded.
🔄 Creating vector store...
✅ Vector DB created and persisted.
🔄 Testing vector store...
✅ Vector store test successful! Found 2 similar documents.
📋 Sample result: -- 10. Q: Show department-wise expense distribution by category
SELECT 
    de.department,
    dc.ca...
🎉 SUCCESS! Vector store created successfully!
✅ You can proceed to the next cells.


# Cell 4: DeepSeek LLM Setup Function


In [4]:
# Step 3: Setup DeepSeek API via OpenRouter
def setup_deepseek_llm():
    """Setup DeepSeek API connection via OpenRouter"""
    # Your OpenRouter API key
    api_key = "YOUR API KEY "
    
    try:
        llm = ChatOpenAI(
            model="deepseek/deepseek-chat",  # OpenRouter model format
            api_key=api_key,
            base_url="https://openrouter.ai/api/v1",  # OpenRouter endpoint
            temperature=0.1,  # Lower temperature for more consistent SQL generation
            max_tokens=512,
            timeout=30,  # Add timeout for reliability
        )
        
        # Test the connection
        print("🔄 Testing OpenRouter connection...")
        test_response = llm.invoke("Hello, can you generate SQL?")
        print("✅ OpenRouter connection successful!")
        print(f"📋 Test response: {test_response.content[:100]}...")
        
        return llm
        
    except Exception as e:
        print(f"❌ Error setting up OpenRouter: {str(e)}")
        print("🔍 Please check:")
        print("   - Your OpenRouter API key is correct")
        print("   - You have credits in your OpenRouter account")
        print("   - DeepSeek model is available on OpenRouter")
        return None

# Test the setup
print("Setting up DeepSeek via OpenRouter...")
print("=" * 50)

llm = setup_deepseek_llm()

if llm:
    print("=" * 50)
    print("🎉 SUCCESS! DeepSeek LLM ready via OpenRouter!")
    print("✅ You can proceed to Cell 5.")
else:
    print("=" * 50)
    print("❌ FAILED! Please check the errors above.")

Setting up DeepSeek via OpenRouter...
🔄 Testing OpenRouter connection...
✅ OpenRouter connection successful!
📋 Test response: Certainly! Here's an example of a simple SQL query that retrieves data from a hypothetical `employee...
🎉 SUCCESS! DeepSeek LLM ready via OpenRouter!
✅ You can proceed to Cell 5.


# Cell 5: RAG Chain Creation Function

In [5]:
# Step 4: Create the RAG chain with DeepSeek API
def create_rag_chain():
    """Create the complete RAG chain with DeepSeek API"""
    
    # Setup vector store and retriever
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = Chroma(
        persist_directory="vector_db",
        embedding_function=embedding_model
    )
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    
    # Setup DeepSeek LLM
    llm = setup_deepseek_llm()
    
    # Enhanced prompt template for better SQL generation
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template="""You are an expert SQL assistant for a campervan database system.

Database Schema and Examples:
{context}

User Question: {question}

Instructions:
- Generate only a valid MySQL SQL query
- Use proper table aliases when joining multiple tables  
- Follow MySQL syntax exactly
- Do not include any explanations or markdown formatting
- Return only the SQL query

SQL Query:"""
    )
    
    # Create the RAG chain
    rag_chain = (
        {
            "context": RunnableLambda(lambda x: x["question"]) | retriever,
            "question": RunnableLambda(lambda x: x["question"])
        }
        | prompt_template
        | llm
        | StrOutputParser()
    )
    
    return rag_chain

# Cell 6: SQL Execution Function

In [6]:
# Step 5: Enhanced SQL execution with error handling
def execute_sql_safely(query, conn):
    """Execute SQL query with error handling"""
    try:
        # Clean the query (remove any markdown formatting)
        clean_query = query.strip()
        if clean_query.startswith("```sql"):
            clean_query = clean_query.replace("```sql", "").replace("```", "").strip()
        
        df = pd.read_sql(clean_query, conn)
        return df, None
    except Exception as e:
        return None, str(e)

# Cell 7: NL2SQL Pipeline Class

In [7]:
# Step 6: Complete NL2SQL pipeline
class NL2SQLPipeline:
    def __init__(self):
        self.conn = conn
        self.rag_chain = None
        
    def initialize(self):
        """Initialize the RAG chain"""
        print("Initializing NL2SQL pipeline...")
        self.rag_chain = create_rag_chain()
        print("Pipeline initialized successfully!")
    
    def query(self, natural_language_question):
        """Convert natural language to SQL and execute"""
        if not self.rag_chain:
            raise ValueError("Pipeline not initialized. Call initialize() first.")
        
        print(f"Processing question: {natural_language_question}")
        
        # Generate SQL using RAG
        generated_sql = self.rag_chain.invoke({"question": natural_language_question})
        print(f"Generated SQL: {generated_sql}")
        
        # Execute SQL
        result_df, error = execute_sql_safely(generated_sql, self.conn)
        
        if error:
            return {
                "sql": generated_sql,
                "data": None,
                "error": error,
                "success": False
            }
        
        return {
            "sql": generated_sql,
            "data": result_df,
            "error": None,
            "success": True
        }

# Cell 8: Utility Functions


In [8]:
# Additional utility functions for better integration
def get_table_schema(conn, table_name):
    """Get table schema information"""
    query = f"DESCRIBE {table_name}"
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        return f"Error getting schema: {e}"

def get_sample_data(conn, table_name, limit=5):
    """Get sample data from table"""
    query = f"SELECT * FROM {table_name} LIMIT {limit}"
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        return f"Error getting sample data: {e}"

# Performance monitoring
def track_query_performance(func):
    """Decorator to track query performance"""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Query executed in {end_time - start_time:.2f} seconds")
        return result
    return wrapper

# Cell 9: Initialize Pipeline (Run Once)


In [9]:
# Setup vector store (run this cell only once to create the vector database)
# Uncomment the line below if running for the first time
#setup_vector_store()

# Initialize the pipeline
pipeline = NL2SQLPipeline()
pipeline.initialize()

Initializing NL2SQL pipeline...


C:\Users\ABHINAVVIKRAM\AppData\Local\Temp\ipykernel_9788\100052839.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


🔄 Testing OpenRouter connection...
✅ OpenRouter connection successful!
📋 Test response: Of course! Could you specify what kind of SQL query you need? For example:  

- **Database schema** ...
Pipeline initialized successfully!


# Cell 10: Test Queries


In [10]:
# Interactive cell for testing individual queries
question = " Find all expense categories that contain the word 'Vehicle'"
result = pipeline.query(question)

if result["success"]:
    print(f"✅ Query successful!")
    print(f"SQL: {result['sql']}")
    print(f"Results: {len(result['data'])} rows")
    display(result['data'])
else:
    print(f"❌ Query failed!")
    print(f"SQL: {result['sql']}")
    print(f"Error: {result['error']}")

Processing question:  Find all expense categories that contain the word 'Vehicle'
Generated SQL: SELECT category_id, category_name, sub_category_name
FROM dim_category
WHERE category_name LIKE '%Vehicle%';
✅ Query successful!
SQL: SELECT category_id, category_name, sub_category_name
FROM dim_category
WHERE category_name LIKE '%Vehicle%';
Results: 6 rows


C:\Users\ABHINAVVIKRAM\AppData\Local\Temp\ipykernel_9788\3252234599.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(clean_query, conn)


,category_id,category_name,sub_category_name
0,1,Vehicle Purchase,Van Acquisition
1,2,Vehicle Purchase,Registration
2,3,Vehicle Conversion,Interior Build
3,4,Vehicle Conversion,Electrical System
4,5,Vehicle Conversion,Plumbing
5,6,Vehicle Maintenance,Service & Repairs


In [16]:
# Interactive cell for testing individual queries
question = " Q: I WANT TOTAL EXPENSES OF YEAR 2024 FROM JANUARY TO DECEMBER & 2025 january to2025 april  "
result = pipeline.query(question)

if result["success"]:
    print(f"✅ Query successful!")
    print(f"SQL: {result['sql']}")
    print(f"Results: {len(result['data'])} rows")
    display(result['data'])
else:
    print(f"❌ Query failed!")
    print(f"SQL: {result['sql']}")
    print(f"Error: {result['error']}")

Processing question:  Q: I WANT TOTAL EXPENSES OF YEAR 2024 FROM JANUARY TO DECEMBER & 2025 january to2025 april  
Generated SQL: ```sql
SELECT 
    dd.year,
    dd.month,
    dd.month_name,
    SUM(fe.amount) as monthly_total,
    COUNT(fe.expense_id) as transaction_count
FROM fact_expenses fe
JOIN dim_date dd ON fe.date_id = dd.date_id
WHERE (dd.year = 2024) OR (dd.year = 2025 AND dd.month <= 4)
GROUP BY dd.year, dd.month, dd.month_name
ORDER BY dd.year, dd.month;
```
✅ Query successful!
SQL: ```sql
SELECT 
    dd.year,
    dd.month,
    dd.month_name,
    SUM(fe.amount) as monthly_total,
    COUNT(fe.expense_id) as transaction_count
FROM fact_expenses fe
JOIN dim_date dd ON fe.date_id = dd.date_id
WHERE (dd.year = 2024) OR (dd.year = 2025 AND dd.month <= 4)
GROUP BY dd.year, dd.month, dd.month_name
ORDER BY dd.year, dd.month;
```
Results: 16 rows


C:\Users\ABHINAVVIKRAM\AppData\Local\Temp\ipykernel_9788\3252234599.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(clean_query, conn)


,year,month,month_name,monthly_total,transaction_count
0,2024,1,January,113615.0,54
1,2024,2,February,112300.0,53
2,2024,3,March,119665.0,57
3,2024,4,April,121220.0,53
4,2024,5,May,123115.0,55
5,2024,6,June,132890.0,55
6,2024,7,July,132125.0,55
7,2024,8,August,134150.0,55
8,2024,9,September,128905.0,46
9,2024,10,October,121440.0,44


# How to Measure Accuracy & Performance
Accuracy Measurement Methods:
# 1. Query Correctness Testing

In [17]:
# Add this to your pipeline for accuracy tracking
def measure_accuracy(test_cases):
    """
    test_cases = [
        {"question": "What are total expenses by department?", 
         "expected_result_count": 5,  # 5 departments
         "expected_columns": ["department", "total_expenses"]},
        # Add more test cases
    ]
    """
    correct_queries = 0
    total_queries = len(test_cases)
    
    for case in test_cases:
        result = pipeline.query(case["question"])
        if result["success"] and len(result["data"]) == case["expected_result_count"]:
            correct_queries += 1
    
    accuracy = (correct_queries / total_queries) * 100
    print(f"Query Accuracy: {accuracy}%")
    return accuracy

# 2. Response Time Benchmarking

In [18]:
import time

def benchmark_pipeline():
    questions = ["Your test questions here"]
    times = []
    
    for q in questions:
        start = time.time()
        result = pipeline.query(q)
        end = time.time()
        times.append(end - start)
    
    avg_time = sum(times) / len(times)
    print(f"Average Response Time: {avg_time:.2f} seconds")
    return avg_time

# 3. Error Rate Tracking

In [19]:
def track_error_rate(test_queries):
    successful = 0
    failed = 0
    
    for query in test_queries:
        result = pipeline.query(query)
        if result["success"]:
            successful += 1
        else:
            failed += 1
    
    success_rate = (successful / (successful + failed)) * 100
    print(f"Success Rate: {success_rate}%")
    return success_rate

Cell 12: Database Schema Exploration

In [ ]:
# Explore database schema
tables = ['dim_date', 'dim_category', 'dim_employee', 'fact_expenses']

for table in tables:
    print(f"\n=== {table.upper()} ===")
    schema = get_table_schema(conn, table)
    print("Schema:")
    display(schema)
    
    print("\nSample Data:")
    sample = get_sample_data(conn, table, 3)
    display(sample)